In [ ]:
!pip install ultralytics

In [9]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load model YOLO
model = YOLO("/content/drive/MyDrive/TA/train4/weights/best.pt")

# Buka video
video_path = "/content/drive/MyDrive/TA/cctv2.mp4"
cap = cv2.VideoCapture(video_path)

# Ambil lebar dan tinggi frame
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

line_y = 351
offset= 10

# Simpan ID kendaraan yang sudah dihitung
counted_ids = set()

# Gunakan tracking agar bisa mendeteksi pergerakan objek
track_history = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Lakukan prediksi pada frame
    results = model.track(frame, persist=True, conf=0.5)

    if results[0].boxes.id is not None:
        for box, track_id in zip(results[0].boxes.xyxy, results[0].boxes.id):
            x1, y1, x2, y2 = map(int, box)  # Koordinat bounding box
            obj_center = (int((x1 + x2) / 2), int((y1 + y2) / 2))  # Titik tengah objek

            # Simpan history posisi object
            if track_id not in track_history:
                track_history[track_id] = []

            track_history[track_id].append(obj_center)

            # Cek apakah kendaraan melewati garis
            if len(track_history[track_id]) > 1:
                prev_y = track_history[track_id][-2][1]  # Posisi Y sebelumnya
                curr_y = obj_center[1]  # Posisi Y sekarang

                # Cek perpindahan melewati garis dari atas ke bawah atau sebaliknya
                if (prev_y < line_y <= curr_y or prev_y > line_y >= curr_y) and track_id not in counted_ids:
                    counted_ids.add(track_id)
                    print(f"Kendaraan dengan ID {track_id} melewati garis!")

            # Gambar bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, str(track_id), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Gambar garis pada frame
    cv2.line(frame, (0, line_y), (frame_width, line_y), (0, 0, 255), 2)

cap.release()
cv2.destroyAllWindows()

print(f"Total kendaraan yang melewati garis: {len(counted_ids)}")



0: 384x640 3 cars, 9.7ms
Speed: 1.7ms preprocess, 9.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 6.6ms
Speed: 1.6ms preprocess, 6.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 6.7ms
Speed: 1.9ms preprocess, 6.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 6.8ms
Speed: 1.6ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 6.9ms
Speed: 1.8ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 6.7ms
Speed: 1.6ms preprocess, 6.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 6.7ms
Speed: 1.6ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 7.1m